In [ ]:
# 模块导入
import pandas as pd

In [ ]:
# 读入txt 每一行的数据形如:1,30.624806,104.136604,1,2014/8/3 21:18:46
# 1:出租车ID 2:维度 3:经度 4:载客情况(1为有,0为无) 5:时间
# 读入txt文件，将每个出租车的数据分别存入一个csv文件中
file_path="20140803_train.txt"
column_names=['taxi_id','latitude','longitude','passenger','time']
# 读入txt文件1
df=pd.read_csv(file_path,sep=',',header=None,names=column_names)


In [ ]:
df.head(5)

# 数据预处理
计算指标:里程利用率、时间利用率、空驶时间

In [ ]:
# 提取出taxi_id为1和2的数据
# df_selected = df[(df['taxi_id']==1) | (df['taxi_id']==2)].copy() # 测试用
df_selected = df

In [ ]:
# 把time列形如:2014/8/3 06:00:53的数据转换成易于处理的形式
df_selected['time']=pd.to_datetime(df_selected['time'],format='%Y/%m/%d %H:%M:%S')

In [ ]:
# 增加一列time_s,表示以秒为单位的时间
df_selected['time_s']=df_selected['time'].dt.hour*3600+df_selected['time'].dt.minute*60+df_selected['time'].dt.second
# 以taxi_id为第一关键字，以time为第二关键字，对数据进行排序
df_selected=df_selected.sort_values(by=['taxi_id','time_s'])

In [ ]:
# 每辆车的GPS数据采取的时间间隔最长和最短值
time_interval_min = df_selected.groupby('taxi_id')['time_s'].diff().min()
time_interval_max = df_selected.groupby('taxi_id')['time_s'].diff().max()

In [ ]:
# 定义一个函数，用来计算两个经纬度之间的距离
from geopy.distance import geodesic

def get_distance(point1, point2):
    """
    计算两个经纬度之间的距离
    :param point1: 起点经纬度 (纬度, 经度)
    :param point2: 终点经纬度 (纬度, 经度)
    :return: 距离（km）
    """
    return geodesic(point1, point2).km

In [ ]:
column_names=['taxi_id','passenger','start_time','end_time','start_latitude','start_longitude','end_latitude','end_longitude','distance']
# 当passenger列为1时，表示载客，为0时表示无客,以此来把轨迹GPS划分为载客轨迹和无客轨迹
# 并记录载客轨迹和无客轨迹的起始时间和终止时间(s)，起始地点和终止地点、轨迹长度(km)(不是位移)
trip_df=pd.DataFrame(columns=column_names)

finished = True # 标记是否已经完成一次轨迹的记录
cur_passenger = 0 # 当前载客情况
cur_distance = 0 # 当前轨迹长度
cur_taxi = 0 # 当前出租车ID
start_latitude = 0 # 轨迹的起始维度
start_longitude = 0 # 轨迹的起始经度
# 合并df_selected中连续的载客数据或无客数据，形成一条轨迹
for index, row in df_selected.iterrows():
    if finished: # 如果已经完成一次轨迹的记录，则开始记录新的轨迹
        # 轨迹的起始时间
        start_time = row['time_s']
        # 轨迹的起始地点
        start_latitude = row['latitude']
        start_longitude = row['longitude']
        # 轨迹的载客情况
        cur_passenger = row['passenger']
        # 轨迹的长度
        cur_distance = 0
        # 轨迹的出租车ID
        cur_taxi = row['taxi_id']
        # 记录完成标志赋值为False
        finished = False
        continue # 跳过本次循环
    if (cur_passenger != row['passenger'] and cur_taxi == row['taxi_id']) or cur_taxi != row['taxi_id']:
        # 如果载客情况发生了变化或者出租车ID发生了变化，则表示本段轨迹记录完成，记录其终止时间、终止地点
        end_time = row['time_s']
        end_latitude = row['latitude']
        end_longitude = row['longitude']
        cur_distance += get_distance((start_latitude, start_longitude), (end_latitude, end_longitude))
        # 将轨迹的信息记录到trip_df中
        trip_df = trip_df.append({'taxi_id': cur_taxi, 'passenger': cur_passenger, 'start_time': start_time, 'end_time': end_time, 'start_longitude': start_longitude, 'start_latitude': start_latitude, 'end_longitude': end_longitude, 'end_latitude': end_latitude, 'distance': cur_distance}, ignore_index=True)
        finished = True
    else: # 如果载客情况没有发生变化，则添加轨迹长度，继续记录轨迹
        cur_distance += get_distance((row['latitude'], row['longitude']), (start_latitude, start_longitude))

# 数据分析
计算目标:每辆车的里程利用率、时间利用率、空驶时间

In [ ]:
# 根据trip_df中的数据，计算每辆出租车的里程利用率、时间利用率、总空驶时间
taxi_df = pd.DataFrame(columns=['taxi_id', 'mileage_utilization', 'time_utilization', 'empty_time'])
for taxi_id in trip_df['taxi_id'].unique():
    # 计算每辆出租车的里程利用率、时间利用率、总空驶时间
    mileage_utilization = trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['distance'].sum() / \
        trip_df[trip_df['taxi_id']==taxi_id]['distance'].sum()
    time_utilization = (trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['end_time'].sum() - \
                        trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==1)]['start_time'].sum()) / \
                            (trip_df[trip_df['taxi_id']==taxi_id]['end_time'].sum() - \
                             trip_df[trip_df['taxi_id']==taxi_id]['start_time'].sum())
    empty_time = trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==0)]['end_time'].sum() - \
        trip_df[(trip_df['taxi_id']==taxi_id) & (trip_df['passenger']==0)]['start_time'].sum()
    # 将计算结果记录到taxi_df中
    taxi_df = taxi_df.append({'taxi_id': taxi_id, 'mileage_utilization': mileage_utilization, 'time_utilization': time_utilization, 'empty_time': empty_time}, ignore_index=True)


计算目标:成都市一天内的打车需求时间分布

计算目标:成都市一天内的打车需求地区分布